Run with the following conda environment: `../../conda_envs/training_env`

In [ ]:
folder = "../../results/"
train_minutes = 5
trace_name_caida = "caida"
trace_name_mawi = "mawi"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pylab as pylab
from pprint import pprint
import pickle
import matplotlib
import matplotlib.transforms as mtrans
import matplotlib.ticker as mtick
from copy import deepcopy
import json
from statistics import fmean
import os
import sys
module_path = os.path.abspath(os.path.join('../../models/utils/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from plots import full_extent

In [ ]:
paths_metrics_caida = [
    f"{folder}/5_pk/all_proto/initial5min_none_0dryrun_caida/",
    f"{folder}/1_pk/all_proto/initial5min_none_0dryrun_caida/",
    f"{folder}/5_pk/all_proto/pheavy_5-20_thr0.6_0dryrun_caida/",
    f"{folder}/1_pk/tcp/initial5min_none_0dryrun_caida/",
    f"{folder}/1_pk/udp/initial5min_none_0dryrun_caida/",
    f"{folder}/1_pk/icmp/initial5min_none_0dryrun_caida/",
]

paths_sizes_caida = [
    f"{folder}/5_pk/all_proto/initial5min_none_0dryrun_caida/",
    f"{folder}/5_pk/all_proto/initial5min_feat_selection+quantization_0dryrun_caida/",
    f"{folder}/5_pk/all_proto/initial5min_pruning+feat_selection+quantization_0dryrun_caida/",
    f"{folder}/5_pk/all_proto/initial5min_ultrapruning+feat_selection+quantization_0dryrun_caida/",
    f"{folder}/5_pk/all_proto/pheavy_5-20_thr0.6_0dryrun_caida/",
]

paths_metrics_mawi = [
    f"{folder}/5_pk/all_proto/initial5min_none_0dryrun_mawi/",
    f"{folder}/1_pk/all_proto/initial5min_none_0dryrun_mawi/",
    f"{folder}/5_pk/all_proto/pheavy_5-20_thr0.6_0dryrun_mawi/",
    f"{folder}/1_pk/tcp/initial5min_none_0dryrun_mawi/",
    f"{folder}/1_pk/udp/initial5min_none_0dryrun_mawi/",
    f"{folder}/1_pk/icmp/initial5min_none_0dryrun_mawi/",
]

In [ ]:
params = {
    'legend.fontsize': 14,
    'legend.title_fontsize': 16,
    'axes.labelsize': 20,
    'axes.titlesize': 22,
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'figure.titlesize': 25,
}
plt.rc('font', size=16)
plt.rc('pdf', fonttype=42)
pylab.rcParams.update(params)
palette = {
    "TCP": "tab:red",
    "UDP": "tab:green",
    "ICMP": "tab:blue",
    "TCP_UDP": "darkorange",
    "ALL_PROTO": "darkorange",

    "Ours 5pk": "brown",
    "Ours 1pk": "darkorange",
    "pHeavy 5pk": "grey",
    "pHeavy 16pk": "black",
    "pHeavy 17pk": "black",
    "pHeavy 18pk": "black",

    "original": "brown",
    "quantized": "blue",
    "pruned+quantized": "darkgreen",
    "aggressivepruned+quantized": "purple",
    "ultrapruned+quantized": "purple",
}

MARKERS = {

    # Protocols
    "ALL_PROTO": "s",
    "TCP_UDP": "s",
    "TCP": "x",
    "UDP": "h",
    "ICMP": "v",

    # k
    "Ours 1pk": "s",
    "Ours 5pk": "x",
    "pHeavy 5pk": ">",
    "pHeavy 16pk": ">",
    "pHeavy 17pk": ">",
    "pHeavy 18pk": ">",

    # Sizes
    "original": "X",
    "quantized": "D",
    "pruned+quantized": "o",
    "aggressivepruned+quantized": "p",
    "ultrapruned+quantized": "p",
}

Load metrics

In [ ]:
metrics_caida = {}
sizes_caida = {}
metrics_mawi = {}
sizes_mawi = {}

for path in paths_metrics_caida + paths_sizes_caida:
    with open(f"{path}minute_APscore_initial_vs_CL.pkl", "rb") as f:
        metrics_caida[path] = pickle.load(f)
    if "pheavy" in path:
        with open(f"{path}model_pheavy.pkl", "rb") as f:
            sizes = pickle.load(f)
            sizes = [v["size"] for v in sizes.values()]
            sizes_caida[path] = {"shrunk": sum(sizes)}
    else:
        with open(f"{path}initial_pipeline_sizes.pkl", "rb") as f:
            sizes_caida[path] = pickle.load(f)

for path in paths_metrics_mawi:
    with open(f"{path}minute_APscore_initial_vs_CL.pkl", "rb") as f:
        metrics_mawi[path] = pickle.load(f)
    if "pheavy" in path:
        with open(f"{path}model_pheavy.pkl", "rb") as f:
            sizes = pickle.load(f)
            sizes = [v["size"] for v in sizes.values()]
            sizes_mawi[path] = {"shrunk": sum(sizes)}
    else:
        with open(f"{path}initial_pipeline_sizes.pkl", "rb") as f:
            sizes_mawi[path] = pickle.load(f)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4.5)) 
(ax_caida_ap, ax_mawi_ap) = axes

for i, path in enumerate(paths_metrics_caida):
    if not "tcp_udp" in path and "1_pk" in path:
        print(path)
        name = path.split("/")[-3]
        if "5_pk" in path:
            n_pk = 5
        if "1_pk" in path:
            n_pk = 1
        lw = 1 if n_pk == 5 else 1
        ls = "-"
        scores_ap = metrics_caida[path]["initial_model_AP"] 
        ax_caida_ap.plot(
            range(train_minutes, train_minutes + len(scores_ap)), 
            scores_ap, 
            label=f"{name.upper()} {n_pk}pk",
            marker=MARKERS[f"{name.upper()}"], markevery=5, 
            c=palette[f"{name.upper()}"], lw=lw, ls=ls,
        )

for i, path in enumerate(paths_metrics_mawi):
    if not "tcp_udp" in path and "1_pk" in path:
        print(path)
        name = path.split("/")[-3]
        if "5_pk" in path:
            n_pk = 5
        if "1_pk" in path:
            n_pk = 1
        lw = 1 if n_pk == 5 else 1
        ls = "-"
        scores_ap = metrics_mawi[path]["initial_model_AP"] 
        ax_mawi_ap.plot(
            range(train_minutes, train_minutes + len(scores_ap)), 
            scores_ap, 
            label=f"{name.upper()} {n_pk}pk",
            marker=MARKERS[f"{name.upper()}"], markevery=5, 
            c=palette[f"{name.upper()}"], lw=lw, ls=ls,
        )

ax_caida_ap.set_ylabel('AP')
ax_caida_ap.set_yticks([0.40, 0.5, 0.60, 0.7, 0.80], ["0.4", "0.5", "0.6", "0.7", "0.8"])
ax_caida_ap.text(51.5, 0.55, 'CAIDA', color='black', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
ax_caida_ap.set_xticks([5, 20, 35, 50], ["", "", "", ""])
ax_caida_ap.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: f"13:{int(x):02d}" if x < 60 else f"14:00"))
plt.setp(ax_caida_ap.get_xticklabels(), rotation=0,)
handles, labels = ax_caida_ap.get_legend_handles_labels()
order = [0, 1, 2, 3]
legend1 = ax_caida_ap.legend(
    [handles[idx] for idx in order], [labels[idx].replace("ALL_PROTO 1pk", "All protos 1pk") for idx in order], 
    loc=(0.25, 0.65), labelspacing=0.25, columnspacing=0.5, ncol=2, framealpha=0., handlelength=1.5
)
fig.add_artist(legend1)

ax_mawi_ap.set_ylabel('AP')
ax_mawi_ap.set_yticks([0.40, 0.5, 0.60, 0.7, 0.80], ["0.4", "0.5", "0.6", "0.7", "0.8"])
ax_mawi_ap.text(52.5, 0.375, 'MAWI', color='black', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
ax_mawi_ap.set_xticks([5, 20, 35, 50])
ax_mawi_ap.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: f"18:{int(x)+45:02d}" if (int(x) + 45) < 60 else f"19:{int(x)+45-60:02d}"))
plt.setp(ax_mawi_ap.get_xticklabels(), rotation=0,)

plt.tight_layout(h_pad=0)
plt.savefig(f'./out/10Aapp_model_protocols_caida_ap_1pk.pdf', format='pdf', dpi=1200)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4.5)) 
(ax_caida_ap, ax_mawi_ap) = axes

for i, path in enumerate(paths_metrics_caida):
    if "tcp_udp" in path or "all_proto" in path:
        print(path)
        name = path.split("/")[-3]

        if "pheavy" in path:
            pk_pheavy = list(metrics_caida[path]["initial_model_AP"][0].keys())
            pk_pheavy = [min(pk_pheavy), max(pk_pheavy)]
            lw = 1
            ls = "-"
            for pk in pk_pheavy:
                scores_ap = [dict[pk] for dict in metrics_caida[path]["initial_model_AP"]]
                ax_caida_ap.plot(
                    range(train_minutes, train_minutes + len(scores_ap)), 
                    scores_ap, 
                    label=f"pHeavy {pk}pk", 
                    marker=MARKERS[f"pHeavy {pk}pk"],  markevery=5,
                    c=palette[f"pHeavy {pk}pk"], ls=ls, lw=lw,
                )
        else:
            if "5_pk" in path:
                n_pk = 5
            if "1_pk" in path:
                n_pk = 1
            lw = 1 if n_pk == 5 else 1
            ls = "-"
            scores_ap = metrics_caida[path]["initial_model_AP"] 
            ax_caida_ap.plot(
                range(train_minutes, train_minutes + len(scores_ap)), 
                scores_ap, 
                label=f"Ours {n_pk}pk",
                marker=MARKERS[f"Ours {n_pk}pk"], markevery=5, 
                c=palette[f"Ours {n_pk}pk"], lw=lw, ls=ls,
            )

for i, path in enumerate(paths_metrics_mawi):
    if "tcp_udp" in path or "all_proto" in path:
        print(path)
        name = path.split("/")[-3]

        if "pheavy" in path:
            pk_pheavy = list(metrics_mawi[path]["initial_model_AP"][0].keys())
            pk_pheavy = [min(pk_pheavy), max(pk_pheavy)]
            lw = 1
            ls = "-"
            for pk in pk_pheavy:
                scores_ap = [dict[pk] for dict in metrics_mawi[path]["initial_model_AP"]]
                ax_mawi_ap.plot(
                    range(train_minutes, train_minutes + len(scores_ap)), 
                    scores_ap, 
                    label=f"pHeavy {pk}pk", 
                    marker=MARKERS[f"pHeavy {pk}pk"],  markevery=5,
                    c=palette[f"pHeavy {pk}pk"], ls=ls, lw=lw,
                )
        else:
            if "5_pk" in path:
                n_pk = 5
            if "1_pk" in path:
                n_pk = 1
            lw = 1 if n_pk == 5 else 1
            ls = "-"
            scores_ap = metrics_mawi[path]["initial_model_AP"] 
            ax_mawi_ap.plot(
                range(train_minutes, train_minutes + len(scores_ap)), 
                scores_ap, 
                label=f"Ours {n_pk}pk",
                marker=MARKERS[f"Ours {n_pk}pk"], markevery=5, 
                c=palette[f"Ours {n_pk}pk"], lw=lw, ls=ls,
            )

ax_caida_ap.set_ylabel('AP')
ax_caida_ap.set_yticks([0.20, 0.40, 0.60, 0.8], ["0.2", "0.4", "0.6", "0.8"])
ax_caida_ap.text(51.5, 0.3, 'CAIDA', color='black', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
ax_caida_ap.set_xticks([5, 20, 35, 50], ["", "", "", ""])
ax_caida_ap.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: f"13:{int(x):02d}" if x < 60 else f"14:00"))
plt.setp(ax_caida_ap.get_xticklabels(), rotation=0,)
handles, labels = ax_caida_ap.get_legend_handles_labels()
order = [0, 1, 2, 3]
legend1 = ax_caida_ap.legend(
    [handles[idx] for idx in order], [labels[idx].replace("pHeavy 17pk", "pHeavy 16/17 pk").replace("pHeavy 16pk", "pHeavy 16/17 pk") for idx in order], 
    loc=(0, 0.2), labelspacing=0.25, columnspacing=0.5, ncol=2, framealpha=0., handlelength=1.5
)
fig.add_artist(legend1)

ax_mawi_ap.set_ylabel('AP')
ax_mawi_ap.set_yticks([0.6, 0.7, 0.8, 0.9, 1.0], ["0.6", "0.7", "0.8", "0.9","1.0"])
ax_mawi_ap.text(52.5, 0.8, 'MAWI', color='black', size=16, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
ax_mawi_ap.set_xticks([5, 20, 35, 50])
ax_mawi_ap.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: f"18:{int(x)+45:02d}" if (int(x) + 45) < 60 else f"19:{int(x)+45-60:02d}"))
plt.setp(ax_mawi_ap.get_xticklabels(), rotation=0,)

plt.tight_layout(h_pad=0)
plt.savefig(f'./out/10Bapp_model_k_caida_mawi_ap.pdf', format='pdf', dpi=1200)
plt.show()